## Imports

In [2]:
from qiskit import *
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from qiskit.providers.aer import QasmSimulator
import qiskit.quantum_info as qi
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector, plot_histogram

import numpy as np
import math
import random

%matplotlib inline

from sklearn.preprocessing import MinMaxScaler

import matplotlib.pyplot as plt
from matplotlib import cm

## Dataset generation

Output of the old generator
```python
[(2,
  array([45, 11, 40, 38]),
  {(0, 2): -15, (0, 3): -9, (1, 2): -9, (1, 3): -14}),
 (2,
  array([10, 37,  9, 46]),
  {(0, 2): -20, (0, 3): -15, (1, 2): -2, (1, 3): -4}),
 (2,
  array([12, 48, 35, 38]),
  {(0, 2): -3, (0, 3): -7, (1, 2): -19, (1, 3): -8}),
 (2,
  array([ 4, 42, 28, 33]),
  {(0, 2): -11, (0, 3): -10, (1, 2): -1, (1, 3): -14}),
 (2,
  array([43, 23, 23, 18]),
  {(0, 2): -3, (0, 3): -2, (1, 2): -16, (1, 3): 0})]
  ```

In [23]:
def create_savings(n_queries, n_plans_per_query):
    savings = {}
    for i in range(n_queries-1):
        for j in range(n_plans_per_query[i]):
            for a in range(i+1, n_queries,1):
                for b in range(n_plans_per_query[a]):
                    for x in n_plans_per_query[i+1:]:
                        for y in range(x):
                            if i == 0:
                                savings[j, y + np.sum(n_plans_per_query[i:a])] = random.randint(-20, 0)
                            else:
                                savings[j + np.sum(n_plans_per_query[0:i]), y + np.sum(n_plans_per_query[i:a])] = random.randint(-20, 0)

    return savings

In [10]:
def create_problems(n_problems, n_queries, n_plans_per_query, cost_min = 0, cost_max = 50, savings_min = -20, savings_max = 0):
    problems = []
    for i in range(n_problems):
        problems.append((n_plans_per_query, np.random.randint(cost_min, cost_max, np.sum(n_plans_per_query)), 
            create_savings(n_queries, n_plans_per_query)))
    return problems

In [14]:
ppq = [4,2,3]

np.sum(ppq[0:2])

6

Problems are generated, but only work, for now, in double combinations -> no three way savings!

In [92]:
problems = create_problems(1, 3, [2,3,2])
problems

[([2, 3, 2],
  array([44, 16, 43, 24, 30, 10, 12]),
  {(0, 2): -3,
   (0, 3): -17,
   (0, 4): -2,
   (0, 5): -20,
   (0, 6): -16,
   (0, 7): -6,
   (1, 2): -1,
   (1, 3): -5,
   (1, 4): -1,
   (1, 5): -6,
   (1, 6): -11,
   (1, 7): -2,
   (2, 3): -4,
   (2, 4): 0,
   (3, 3): -7,
   (3, 4): -13,
   (4, 3): -2,
   (4, 4): -9})]

We now generate the combinational bitstrings that are possible

In [91]:
n_qubits = np.sum(problems[0][0])
binary_string = []
for i, v in enumerate(problems[0][0]):
    if i == 0:
        for j in range(v):
            binary_string.append('0'*j + '1' + '0'*(v-j-1))
    else:
        copy = []
        for x in binary_string:
            for j in range(v):
                copy.append(x + '0'*j + '1' + '0'*(v-j-1))
        binary_string = copy


print(binary_string)


['101010', '101001', '100110', '100101', '011010', '011001', '010110', '010101']


Now we generate the circuits...